# Генерация синтетического датасета
В этом ноутбуке мы разберем генерацию синтетического датасета с помощью библиотеки `giskard`
Библиотека `giskard` берет каждый документ в нашей векторной БД и генерирует вопросы по каждому документу с помощью 6 разных способов. (Мы будем использовать 5)
Этот датасет мы будем использовать в дальнейшей оценке нашего RAG

In [ ]:
!pip install "giskard[llm]" pandas gigachain-community unstructured

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Загружаем документы

In [2]:
from giskard.rag import generate_testset, KnowledgeBase
from langchain_community.document_loaders import UnstructuredMarkdownLoader
import os
import tqdm
import pandas as pd
markdown_path = "../db_builder/mdx_docs/"

docuemnts = []
# Iterate all mdx file in subdirectories
for root, dirs, files in tqdm.tqdm(os.walk(markdown_path)):
    for file in files:
        if file.endswith(".mdx"):
            loader = UnstructuredMarkdownLoader(os.path.join(root, file))
            docuemnts.append(loader.load())
print(f"Total documents: {len(docuemnts)}")

18it [00:06,  2.82it/s]

Total documents: 53


Нарезаем их

In [3]:
from langchain.text_splitter import MarkdownTextSplitter

docs_list = [item for sublist in docuemnts for item in sublist]
text_splitter = MarkdownTextSplitter(chunk_size=16000, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs_list)
print(f"Documents splited. Count: {len(doc_splits)}")

Documents splited. Count: 56


In [4]:
df = pd.DataFrame([d.page_content for d in doc_splits[:]], columns=["text"])
df[:10]

,text
0,Как обновить GigaChain?\n\nВыполните команду b...
1,Общие понятия\n\nclient_id - идентификатор про...
2,GigaChat. Python-библиотека для GigaChain\n\nБ...
3,🦜️🔗 GigaChain (GigaChat + LangChain)\n\nБиблио...
4,Главная\n\nБаза знаний\n\nМедитация от страха ...
5,ИНТЕГРИРУЙТЕ ИИ В СВОЙ БИЗНЕС ================...
6,"напарник в работе, учёбе и дома ==============..."
7,"Не работает GigaChat? Проверьте, что отключен ..."
8,Вход в GigaChat\n\nПосле этого вы увидите окно...
9,Главная\n\nБаза знаний\n\nБаза знаний\n\nБАЗА ...


In [5]:
import giskard
giskard.llm.set_llm_model("gpt-4o")

Генерируем датасет

In [6]:
from giskard.rag.question_generators import complex_questions, double_questions, distracting_questions, situational_questions, simple_questions
knowledge_base = KnowledgeBase.from_pandas(df)

testset = generate_testset(
    knowledge_base,
    question_generators=[simple_questions, complex_questions, double_questions, distracting_questions, situational_questions],
    num_questions=75,
    language='ru',  # optional, we'll auto detect if not provided
    agent_description="Агент по технической поддержке большой языковой модели GigaChat и фреймворка GigaChain", # helps generating better questions
)

2024-09-17 14:23:32,821 pid:59036 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-09-17 14:23:49,921 pid:59036 MainThread giskard.rag  INFO     Found 5 topics in the knowledge base.


Generating questions:   0%|          | 0/75 [00:00<?, ?it/s]

Сохраняем датасет

In [7]:
df = testset.to_pandas()

In [8]:
df[:10]

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
ff91f4fa-c316-4ae2-91e5-19a8b41cdbff,Какой тариф подключается по умолчанию для новы...,Каждому новому пользователю по умолчанию подкл...,Document 10: lastUpdateDate: 19 июля 2024\n\ni...,[],"{'question_type': 'simple', 'seed_document_id'..."
0b23c7a3-0910-4442-8b46-edbb9fbee57b,Как долго действителен токен доступа для GigaC...,Токен действителен в течение 30 минут.,Document 15: lastUpdateDate: 24 июля 2024\n\nБ...,[],"{'question_type': 'simple', 'seed_document_id'..."
1e15bd11-6e4d-475d-b35b-9f68bf4ea9c9,Какое значение имеет поле 'index' в ответе на ...,Индекс объекта с векторным представлением (пол...,"Document 55: id: post-embeddings title: ""Созда...",[],"{'question_type': 'simple', 'seed_document_id'..."
d91be405-ba32-42d5-b187-a35f92c51e19,Какой пример промпта используется для исправле...,"json { ""model"": ""GigaChat"", ""messages"": [ { ""r...",Document 25: lastUpdateDate: 4 апреля 2024\n\n...,[],"{'question_type': 'simple', 'seed_document_id'..."
ad302d4c-2c4a-4527-b495-9c71e02c9d7f,Какой URL используется для обращения к моделям...,Для обращения к моделям в раннем доступе испол...,"Document 54: id: get-models title: ""Получить с...",[],"{'question_type': 'simple', 'seed_document_id'..."
c26d5a7d-5d67-42b6-a430-13a69bdd142f,Что такое эмбеддинг и для чего он используется?,Эмбеддинг (англ. embedding) — это вектор в вид...,"Document 55: id: post-embeddings title: ""Созда...",[],"{'question_type': 'simple', 'seed_document_id'..."
f599aecb-5b75-4827-a9a4-0b70707c0cea,Как можно создать аудиомедитацию в GigaChat?,"Чтобы создать медитацию с помощью GigaChat, не...",Document 4: Главная\n\nБаза знаний\n\nМедитаци...,[],"{'question_type': 'simple', 'seed_document_id'..."
54411bb2-4593-4181-8b74-609d77459600,Какая структура промпта используется для генер...,"json { ""model"": ""GigaChat"", ""messages"": [ { ""r...",Document 18: lastUpdateDate: 4 апреля 2024\n\n...,[],"{'question_type': 'simple', 'seed_document_id'..."
893510b2-dc07-40b1-9f66-23cfe8bf9af8,Какой метод используется для подсчета количест...,Метод POST используется для подсчета количеств...,"Document 52: id: post-tokens-count title: ""Под...",[],"{'question_type': 'simple', 'seed_document_id'..."


In [9]:
df.to_csv("dataset.csv")

Ура! Мы сделали синтетический датасет. Теперь нужно его просмотреть, почистить и переходить к следующим шагам:
1. [Загрузка датасета в GigaLogger](2_gigalogger_create_dataset.ipynb)
2. [Прогон датасета и получение оценок ответам](3_evaluation.ipynb)